# CMIP5: Processing Precip Amounts, 2041-2070

This notebook provides for preliminary processing of precipitation amounts primarily to break up the enormous wet days archive. Want to break this into months and save two different DataFrames:

1. Wet Day Counts
2. Precipitation depths

Then we can proceed to distribution fitting.

Go through a month at a time and create the monthly DataFrame for analysis and then output it for distribution fitting in R.

After outputting then analyze the various DataFrames graphically and output some summary statistics to save from doing this in R.

From Wilks & Wilby (1999)
*"Most stochastic weather generators make the assumption that precipitation amounts
on wet days are independent, and follow the same distribution. Allowing different
probability distributions for precipitation amounts depending on that day’s position in
a wet spell (e.g., the mean rainfall on a wet day following a wet day might be greater
than on a wet day following a dry day) has been considered by Katz (1977), Buishand
(1977; 1978), Chin and Miller (1980) and Wilks (1999a), but allowing this extra
complexity often makes little difference to the result. Similarly, the autocorrelation
between successive nonzero precipitation amounts in daily series is sometimes (statistically) significantly different from zero, but is typically quite small and usually of little
practical importance (Katz, 1977; Buishand, 1977; 1978; Foufoula-Georgiou and
Lettenmaier, 1987). In contrast, accounting for serial correlation of nonzero precipitation
amounts is essential if the precipitation model has an hourly (or smaller) rather than a
daily time step (Katz and Parlange, 1995)."*

In [1]:
from IPython.display import display, HTML
import os
import numpy as np
import pandas as pd
import datetime as dt
import geopandas as gpd
from copy import deepcopy
import re

In [2]:
OUT_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Da' \
          r'ta\JNotes\Processed\CMIP5\CMIP5_2041_WetDays'

Have a DataFrame saved as a pickle file of all of the wet days. Use this as the base for the calculations in this notebook

In [3]:
IN_LOCAP = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Da' \
            r'ta\JNotes\Processed\CMIP5\WetDays_LOCA_2041-2070.pickle'
IN_BCCAP = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Da' \
            r'ta\JNotes\Processed\CMIP5\WetDays_BCCA_2041-2070.pickle'

In [4]:
LOWetDF = pd.read_pickle( IN_LOCAP )

In [5]:
display( HTML( LOWetDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
0,M10_1,1,10,2041,1,5,3,8.592035,2.403758,0.411038,5.777239,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,1,10,2041,1,10,5,27.664704,0.771742,0.640021,4.673357,12.168321,9.411263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,1,10,2041,1,19,1,2.929670,2.929670,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,1,10,2041,1,27,3,12.296292,1.277153,10.359477,0.659662,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,1,10,2041,2,4,1,1.663593,1.663593,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
BCWetDF = pd.read_pickle( IN_BCCAP )

In [7]:
display( HTML( BCWetDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47,Day_48,Day_49,Day_50,Day_51,Day_52,Day_53,Day_54,Day_55,Day_56,Day_57,Day_58,Day_59,Day_60,Day_61,Day_62,Day_63,Day_64,Day_65,Day_66,Day_67,Day_68,Day_69,Day_70
0,M100_169,169,100,2041,1,6,1,1.199132,1.199132,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,169,100,2041,1,16,2,0.853701,0.490519,0.363182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,169,100,2041,1,22,2,6.386087,2.231227,4.154860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,169,100,2041,1,29,2,0.720605,0.447154,0.273450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,169,100,2041,2,2,1,0.266768,0.266768,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
ExModNum = lambda GId: int( (re.match( r'M(.*[0-9])_(.*[0-9])', GId )).group(1) )
ExGridNum = lambda GId: int( (re.match( r'M(.*[0-9])_(.*[0-9])', GId )).group(2) )

## Monthly Data

Now will split this DataFrame up by month, keeping the grid cell id, and only the other needed columns

In [9]:
LOAllCols = list( LOWetDF.columns )
LOAllCols[:7]

['MGrid_Id', 'Grid_Id', 'Mod_Id', 'Year', 'Month', 'Day', 'Wet_Count']

In [10]:
BCAllCols = list( BCWetDF.columns )
BCAllCols[:7]

['MGrid_Id', 'Grid_Id', 'Mod_Id', 'Year', 'Month', 'Day', 'Wet_Count']

In [11]:
LORootCols = deepcopy( LOAllCols[:7] )
LODayCols = deepcopy( LOAllCols[6:] )

In [12]:
BCRootCols = deepcopy( BCAllCols[:7] )
BCDayCols = deepcopy( BCAllCols[6:] )

In [13]:
LOMaxDays = LOWetDF['Wet_Count'].max()
LOMaxDays

47

In [14]:
BCMaxDays = BCWetDF['Wet_Count'].max()
BCMaxDays

70

In [15]:
StartInd = 3

Go through each month and split out the wet days by month and output these for further distribution fitting in R.

### Jan

Counts

In [16]:
LOJanDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 1].copy()

In [17]:
display( HTML( LOJanDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
0,M10_1,1,10,2041,1,5,3
1,M10_1,1,10,2041,1,10,5
2,M10_1,1,10,2041,1,19,1
3,M10_1,1,10,2041,1,27,3
36,M10_1,1,10,2042,1,9,1


In [18]:
BCJanDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 1].copy()

In [19]:
display( HTML( BCJanDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
0,M100_169,169,100,2041,1,6,1
1,M100_169,169,100,2041,1,16,2
2,M100_169,169,100,2041,1,22,2
3,M100_169,169,100,2041,1,29,2
64,M100_169,169,100,2042,1,2,1


In [20]:
LOJanDF['Grid_Id'] = LOJanDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOJanDF['Model_Id'] = LOJanDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCJanDF['Grid_Id'] = BCJanDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCJanDF['Model_Id'] = BCJanDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [21]:
display( HTML( LOJanDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
0,M10_1,1,10,2041,1,5,3,10
1,M10_1,1,10,2041,1,10,5,10
2,M10_1,1,10,2041,1,19,1,10
3,M10_1,1,10,2041,1,27,3,10
36,M10_1,1,10,2042,1,9,1,10


In [22]:
display( HTML( BCJanDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
0,M100_169,169,100,2041,1,6,1,100
1,M100_169,169,100,2041,1,16,2,100
2,M100_169,169,100,2041,1,22,2,100
3,M100_169,169,100,2041,1,29,2,100
64,M100_169,169,100,2042,1,2,1,100


In [23]:
LOJanPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOJanDF.to_pickle( LOJanPCKF )
BCJanPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCJanDF.to_pickle( BCJanPCKF )

In [24]:
LOJanDF = LOJanDF.reset_index()
LOJanFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOJanDF.to_feather( LOJanFeatherF )
BCJanDF = BCJanDF.reset_index()
BCJanFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCJanDF.to_feather( BCJanFeatherF )

Depths

In [25]:
LOJanDF = LOWetDF[LOWetDF['Month'] == 1].copy()
BCJanDF = BCWetDF[BCWetDF['Month'] == 1].copy()

In [26]:
display( HTML( LOJanDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
0,M10_1,1,10,2041,1,5,3,8.592035,2.403758,0.411038,5.777239,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,1,10,2041,1,10,5,27.664704,0.771742,0.640021,4.673357,12.168321,9.411263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,1,10,2041,1,19,1,2.929670,2.929670,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,1,10,2041,1,27,3,12.296292,1.277153,10.359477,0.659662,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,M10_1,1,10,2042,1,9,1,0.954551,0.954551,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
LOJanMaxCnt = LOJanDF['Wet_Count'].max()
LOJanMaxCnt

14

In [28]:
BCJanMaxCnt = BCJanDF['Wet_Count'].max()
BCJanMaxCnt

16

In [29]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [30]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [31]:
for jJ in range(LOJanMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [32]:
for jJ in range(BCJanMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [33]:
LOJanDF = LOJanDF[LOCurCols].copy()
BCJanDF = BCJanDF[BCCurCols].copy()

In [34]:
LOJanDF.reset_index(drop=True, inplace=True)
BCJanDF.reset_index(drop=True, inplace=True)

In [35]:
display( HTML( LOJanDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12
0,M10_1,1,5,3,8.592035,2.403758,0.411038,5.777239,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,1,10,5,27.664704,0.771742,0.640021,4.673357,12.168321,9.411263,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,1,19,1,2.929670,2.929670,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,1,27,3,12.296292,1.277153,10.359477,0.659662,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,1,9,1,0.954551,0.954551,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,1,11,3,12.753678,8.932452,3.096299,0.724927,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,1,18,2,2.699552,0.948289,1.751263,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,1,23,2,4.073620,1.584237,2.489383,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,1,30,1,0.208908,0.208908,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,1,13,1,2.486454,2.486454,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
display( HTML( BCJanDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14
0,M100_169,1,6,1,1.199132,1.199132,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,1,16,2,0.853701,0.490519,0.363182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,1,22,2,6.386087,2.231227,4.154860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,1,29,2,0.720605,0.447154,0.273450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,1,2,1,0.251188,0.251188,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,1,9,2,3.479766,1.715555,1.764211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,1,23,1,0.548875,0.548875,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,1,10,2,12.474728,9.882177,2.592550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,1,13,2,3.898262,0.473524,3.424738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,1,29,1,0.244970,0.244970,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
LOJanTotDays = LOJanDF['Wet_Count'].sum()
LOJanTotDays

1627903

In [38]:
BCJanTotDays = BCJanDF['Wet_Count'].sum()
BCJanTotDays

1295352

Setup the arrays that will use to store all of the wet day precipitation depths

In [39]:
GridIDs = np.zeros( LOJanTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOJanTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOJanTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [40]:
LONumJanWet = len( LOJanDF )
LONumJanWet

912274

In [41]:
iCnt = 0
for iI in range(LONumJanWet):
    cRow = LOJanDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [42]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [43]:
PDDepth.min(), PDDepth.max()

(0.20000003, 140.5188)

In [44]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOJanDayDF = pd.DataFrame( data=DataDict )

In [45]:
display( HTML( LOJanDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.627903e+06,1.627903e+06,1.627903e+06
mean,8.629758e+01,3.203680e+01,4.359602e+00
std,4.918321e+01,1.849162e+01,7.587882e+00
min,1.000000e+00,1.000000e+00,2.000000e-01
25%,4.300000e+01,1.600000e+01,1.000000e+00
50%,8.800000e+01,3.200000e+01,2.000000e+00
75%,1.300000e+02,4.800000e+01,4.000000e+00
max,1.680000e+02,6.400000e+01,1.405188e+02


Now want to save the January DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [46]:
LOJanPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOJanDayDF.to_pickle( LOJanPCKF )
LOJanFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOJanDayDF.to_feather( LOJanFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [47]:
GridIDs = np.zeros( BCJanTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCJanTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCJanTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [48]:
BCNumJanWet = len( BCJanDF )
BCNumJanWet

675263

In [49]:
iCnt = 0
for iI in range(BCNumJanWet):
    cRow = BCJanDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [50]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [51]:
PDDepth.min(), PDDepth.max()

(0.20000184, 123.538994)

In [52]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCJanDayDF = pd.DataFrame( data=DataDict )

In [53]:
display( HTML( BCJanDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.295352e+06,1.295352e+06,1.295352e+06
mean,1.895809e+02,1.298405e+02,3.737265e+00
std,1.212147e+01,3.842621e+01,6.046239e+00
min,1.690000e+02,6.500000e+01,2.000018e-01
25%,1.790000e+02,9.700000e+01,1.000000e+00
50%,1.890000e+02,1.290000e+02,2.000000e+00
75%,2.000000e+02,1.640000e+02,3.787182e+00
max,2.100000e+02,1.960000e+02,1.235390e+02


Now want to save the January DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [54]:
BCJanPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCJanDayDF.to_pickle( BCJanPCKF )
BCJanFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jan_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCJanDayDF.to_feather( BCJanFeatherF )

### Feb

Counts

In [55]:
LOFebDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 2].copy()

In [56]:
display( HTML( LOFebDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
4,M10_1,1,10,2041,2,4,1
5,M10_1,1,10,2041,2,12,8
6,M10_1,1,10,2041,2,24,1
41,M10_1,1,10,2042,2,3,1
42,M10_1,1,10,2042,2,9,2


In [57]:
BCFebDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 2].copy()

In [58]:
display( HTML( BCFebDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
4,M100_169,169,100,2041,2,2,1
5,M100_169,169,100,2041,2,12,2
6,M100_169,169,100,2041,2,16,1
7,M100_169,169,100,2041,2,18,1
8,M100_169,169,100,2041,2,20,2


In [59]:
LOFebDF['Grid_Id'] = LOFebDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOFebDF['Model_Id'] = LOFebDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCFebDF['Grid_Id'] = BCFebDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCFebDF['Model_Id'] = BCFebDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [60]:
display( HTML( LOFebDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
4,M10_1,1,10,2041,2,4,1,10
5,M10_1,1,10,2041,2,12,8,10
6,M10_1,1,10,2041,2,24,1,10
41,M10_1,1,10,2042,2,3,1,10
42,M10_1,1,10,2042,2,9,2,10


In [61]:
display( HTML( BCFebDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
4,M100_169,169,100,2041,2,2,1,100
5,M100_169,169,100,2041,2,12,2,100
6,M100_169,169,100,2041,2,16,1,100
7,M100_169,169,100,2041,2,18,1,100
8,M100_169,169,100,2041,2,20,2,100


In [62]:
LOFebPCKF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOFebDF.to_pickle( LOFebPCKF )
BCFebPCKF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCFebDF.to_pickle( BCFebPCKF )

In [63]:
LOFebDF = LOFebDF.reset_index()
LOFebFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOFebDF.to_feather( LOFebFeatherF )
BCFebDF = BCFebDF.reset_index()
BCFebFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCFebDF.to_feather( BCFebFeatherF )

Depths

In [64]:
LOFebDF = LOWetDF[LOWetDF['Month'] == 2].copy()
BCFebDF = BCWetDF[BCWetDF['Month'] == 2].copy()

In [65]:
display( HTML( LOFebDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
4,M10_1,1,10,2041,2,4,1,1.663593,1.663593,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,1,10,2041,2,12,8,28.847171,0.213769,8.460535,10.907716,3.11058,1.839117,3.637815,0.47319,0.204448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,1,10,2041,2,24,1,9.711517,9.711517,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,M10_1,1,10,2042,2,3,1,3.806781,3.806781,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,M10_1,1,10,2042,2,9,2,1.492044,0.682269,0.809775,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
LOFebMaxCnt = LOFebDF['Wet_Count'].max()
LOFebMaxCnt

18

In [67]:
BCFebMaxCnt = BCFebDF['Wet_Count'].max()
BCFebMaxCnt

21

In [68]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [69]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [70]:
for jJ in range(LOFebMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [71]:
for jJ in range(BCFebMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [72]:
LOFebDF = LOFebDF[LOCurCols].copy()
BCFebDF = BCFebDF[BCCurCols].copy()

In [73]:
LOFebDF.reset_index(drop=True, inplace=True)
BCFebDF.reset_index(drop=True, inplace=True)

In [74]:
display( HTML( LOFebDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16
0,M10_1,2,4,1,1.663593,1.663593,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,2,12,8,28.847171,0.213769,8.460535,10.907716,3.110580,1.839117,3.637815,0.47319,0.204448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,2,24,1,9.711517,9.711517,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,2,3,1,3.806781,3.806781,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,2,9,2,1.492044,0.682269,0.809775,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,2,12,1,0.440870,0.440870,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,2,18,1,3.629009,3.629009,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,2,21,1,6.137404,6.137404,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,2,27,1,0.672557,0.672557,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,2,12,5,29.748754,0.559738,0.619743,19.312477,8.899035,0.357760,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
display( HTML( BCFebDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19
0,M100_169,2,2,1,0.266768,0.266768,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,2,12,2,0.566008,0.223375,0.342633,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,2,16,1,1.123206,1.123206,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,2,18,1,0.906308,0.906308,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,2,20,2,1.709195,0.254280,1.454915,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,2,2,1,0.639732,0.639732,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,2,4,1,0.239603,0.239603,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,2,6,1,0.348821,0.348821,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,2,15,3,5.728009,3.633435,1.358035,0.736538,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,2,3,4,17.028648,5.586631,0.698066,7.311686,3.432265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
LOFebTotDays = LOFebDF['Wet_Count'].sum()
LOFebTotDays

1861981

In [77]:
BCFebTotDays = BCFebDF['Wet_Count'].sum()
BCFebTotDays

1343728

Setup the arrays that will use to store all of the wet day precipitation depths

In [78]:
GridIDs = np.zeros( LOFebTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOFebTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOFebTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [79]:
LONumFebWet = len( LOFebDF )
LONumFebWet

962122

In [80]:
iCnt = 0
for iI in range(LONumFebWet):
    cRow = LOFebDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [81]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [82]:
PDDepth.min(), PDDepth.max()

(0.20001052, 245.33136)

In [83]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOFebDayDF = pd.DataFrame( data=DataDict )

In [84]:
display( HTML( LOFebDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.861981e+06,1.861981e+06,1.861981e+06
mean,8.618291e+01,3.170604e+01,6.097047e+00
std,4.918692e+01,1.833406e+01,1.147900e+01
min,1.000000e+00,1.000000e+00,2.000105e-01
25%,4.400000e+01,1.600000e+01,1.000000e+00
50%,8.700000e+01,3.200000e+01,2.000000e+00
75%,1.300000e+02,4.700000e+01,5.000000e+00
max,1.680000e+02,6.400000e+01,2.453314e+02


Now want to save the Feb DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [85]:
LOFebPCKF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOFebDayDF.to_pickle( LOFebPCKF )
LOFebFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOFebDayDF.to_feather( LOFebFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [86]:
GridIDs = np.zeros( BCFebTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCFebTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCFebTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [87]:
BCNumFebWet = len( BCFebDF )
BCNumFebWet

676723

In [88]:
iCnt = 0
for iI in range(BCNumFebWet):
    cRow = BCFebDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [89]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [90]:
PDDepth.min(), PDDepth.max()

(0.20000128, 206.03667)

In [91]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCFebDayDF = pd.DataFrame( data=DataDict )

In [92]:
display( HTML( BCFebDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.343728e+06,1.343728e+06,1.343728e+06
mean,1.896432e+02,1.290328e+02,4.463125e+00
std,1.212628e+01,3.854934e+01,7.904492e+00
min,1.690000e+02,6.500000e+01,2.000013e-01
25%,1.790000e+02,9.500000e+01,1.000000e+00
50%,1.890000e+02,1.270000e+02,2.000000e+00
75%,2.000000e+02,1.640000e+02,4.000000e+00
max,2.100000e+02,1.960000e+02,2.060367e+02


Now want to save the Feb DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [93]:
BCFebPCKF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCFebDayDF.to_pickle( BCFebPCKF )
BCFebFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Feb_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCFebDayDF.to_feather( BCFebFeatherF )

### Mar

Counts

In [94]:
LOMarDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 3].copy()

In [95]:
display( HTML( LOMarDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
7,M10_1,1,10,2041,3,31,1
47,M10_1,1,10,2042,3,16,1
48,M10_1,1,10,2042,3,22,1
84,M10_1,1,10,2043,3,23,3
116,M10_1,1,10,2044,3,6,1


In [96]:
BCMarDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 3].copy()

In [97]:
display( HTML( BCMarDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
9,M100_169,169,100,2041,3,6,2
10,M100_169,169,100,2041,3,11,2
11,M100_169,169,100,2041,3,20,2
12,M100_169,169,100,2041,3,25,3
71,M100_169,169,100,2042,3,2,3


In [98]:
LOMarDF['Grid_Id'] = LOMarDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOMarDF['Model_Id'] = LOMarDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCMarDF['Grid_Id'] = BCMarDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCMarDF['Model_Id'] = BCMarDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [99]:
display( HTML( LOMarDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
7,M10_1,1,10,2041,3,31,1,10
47,M10_1,1,10,2042,3,16,1,10
48,M10_1,1,10,2042,3,22,1,10
84,M10_1,1,10,2043,3,23,3,10
116,M10_1,1,10,2044,3,6,1,10


In [100]:
display( HTML( BCMarDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
9,M100_169,169,100,2041,3,6,2,100
10,M100_169,169,100,2041,3,11,2,100
11,M100_169,169,100,2041,3,20,2,100
12,M100_169,169,100,2041,3,25,3,100
71,M100_169,169,100,2042,3,2,3,100


In [101]:
LOMarPCKF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOMarDF.to_pickle( LOMarPCKF )
BCMarPCKF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCMarDF.to_pickle( BCMarPCKF )

In [102]:
LOMarDF = LOMarDF.reset_index()
LOMarFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOMarDF.to_feather( LOMarFeatherF )
BCMarDF = BCMarDF.reset_index()
BCMarFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCMarDF.to_feather( BCMarFeatherF )

Depths

In [103]:
LOMarDF = LOWetDF[LOWetDF['Month'] == 3].copy()
BCMarDF = BCWetDF[BCWetDF['Month'] == 3].copy()

In [104]:
display( HTML( LOMarDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
7,M10_1,1,10,2041,3,31,1,0.643898,0.643898,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,M10_1,1,10,2042,3,16,1,0.772882,0.772882,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,M10_1,1,10,2042,3,22,1,3.642475,3.642475,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84,M10_1,1,10,2043,3,23,3,12.235369,6.043658,3.742771,2.448939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,M10_1,1,10,2044,3,6,1,2.388120,2.388120,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
LOMarMaxCnt = LOMarDF['Wet_Count'].max()
LOMarMaxCnt

16

In [106]:
BCMarMaxCnt = BCMarDF['Wet_Count'].max()
BCMarMaxCnt

25

In [107]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [108]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [109]:
for jJ in range(LOMarMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [110]:
for jJ in range(BCMarMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [111]:
LOMarDF = LOMarDF[LOCurCols].copy()
BCMarDF = BCMarDF[BCCurCols].copy()

In [112]:
LOMarDF.reset_index(drop=True, inplace=True)
BCMarDF.reset_index(drop=True, inplace=True)

In [113]:
display( HTML( LOMarDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14
0,M10_1,3,31,1,0.643898,0.643898,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,3,16,1,0.772882,0.772882,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,3,22,1,3.642475,3.642475,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,3,23,3,12.235369,6.043658,3.742771,2.448939,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,3,6,1,2.388120,2.388120,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,3,9,4,7.282198,0.892518,4.397008,1.267350,0.725321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,3,19,1,0.365443,0.365443,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,3,22,3,4.593135,1.864578,0.817469,1.911088,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,3,13,1,0.558517,0.558517,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,3,7,2,3.128615,0.248707,2.879908,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
display( HTML( BCMarDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23
0,M100_169,3,6,2,4.696287,2.940968,1.755319,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,3,11,2,1.804604,0.583943,1.220661,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,3,20,2,20.255838,1.826267,18.429571,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,3,25,3,7.226693,1.146659,5.356657,0.723377,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,3,2,3,13.202728,0.520067,12.314909,0.367752,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,3,13,3,10.271235,1.136092,0.232293,8.902851,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,3,21,2,1.828208,1.555166,0.273041,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,3,25,4,15.237706,0.458970,4.293722,4.812133,5.672880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,3,11,4,3.088512,0.483332,0.952580,1.444596,0.208004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,3,25,1,1.002505,1.002505,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
LOMarTotDays = LOMarDF['Wet_Count'].sum()
LOMarTotDays

1921199

In [116]:
BCMarTotDays = BCMarDF['Wet_Count'].sum()
BCMarTotDays

1503625

Setup the arrays that will use to store all of the wet day precipitation depths

In [117]:
GridIDs = np.zeros( LOMarTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOMarTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOMarTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [118]:
LONumMarWet = len( LOMarDF )
LONumMarWet

1019182

In [119]:
iCnt = 0
for iI in range(LONumMarWet):
    cRow = LOMarDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [120]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [121]:
PDDepth.min(), PDDepth.max()

(0.20000596, 250.80096)

In [122]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOMarDayDF = pd.DataFrame( data=DataDict )

In [123]:
display( HTML( LOMarDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.921199e+06,1.921199e+06,1.921199e+06
mean,8.767955e+01,3.172254e+01,5.727729e+00
std,4.853402e+01,1.846503e+01,1.034103e+01
min,1.000000e+00,1.000000e+00,2.000060e-01
25%,4.600000e+01,1.600000e+01,1.000000e+00
50%,9.000000e+01,3.200000e+01,2.000000e+00
75%,1.300000e+02,4.800000e+01,5.033077e+00
max,1.680000e+02,6.400000e+01,2.508010e+02


Now want to save the Mar DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [124]:
LOMarPCKF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOMarDayDF.to_pickle( LOMarPCKF )
LOMarFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOMarDayDF.to_feather( LOMarFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [125]:
GridIDs = np.zeros( BCMarTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCMarTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCMarTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [126]:
BCNumMarWet = len( BCMarDF )
BCNumMarWet

768584

In [127]:
iCnt = 0
for iI in range(BCNumMarWet):
    cRow = BCMarDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [128]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [129]:
PDDepth.min(), PDDepth.max()

(0.2000008, 244.0524)

In [130]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCMarDayDF = pd.DataFrame( data=DataDict )

In [131]:
display( HTML( BCMarDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.503625e+06,1.503625e+06,1.503625e+06
mean,1.895252e+02,1.292555e+02,4.598876e+00
std,1.211154e+01,3.845153e+01,8.581955e+00
min,1.690000e+02,6.500000e+01,2.000008e-01
25%,1.790000e+02,9.600000e+01,1.000000e+00
50%,1.890000e+02,1.280000e+02,2.000000e+00
75%,2.000000e+02,1.630000e+02,4.000000e+00
max,2.100000e+02,1.960000e+02,2.440524e+02


Now want to save the Mar DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [132]:
BCMarPCKF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCMarDayDF.to_pickle( BCMarPCKF )
BCMarFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Mar_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCMarDayDF.to_feather( BCMarFeatherF )

### Apr

Counts

In [133]:
LOAprDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 4].copy()

In [134]:
display( HTML( LOAprDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
8,M10_1,1,10,2041,4,9,2
9,M10_1,1,10,2041,4,17,2
10,M10_1,1,10,2041,4,22,1
11,M10_1,1,10,2041,4,28,2
49,M10_1,1,10,2042,4,4,2


In [135]:
BCAprDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 4].copy()

In [136]:
display( HTML( BCAprDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
13,M100_169,169,100,2041,4,1,2
14,M100_169,169,100,2041,4,5,3
15,M100_169,169,100,2041,4,10,3
16,M100_169,169,100,2041,4,14,2
17,M100_169,169,100,2041,4,18,4


In [137]:
LOAprDF['Grid_Id'] = LOAprDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOAprDF['Model_Id'] = LOAprDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCAprDF['Grid_Id'] = BCAprDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCAprDF['Model_Id'] = BCAprDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [138]:
display( HTML( LOAprDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
8,M10_1,1,10,2041,4,9,2,10
9,M10_1,1,10,2041,4,17,2,10
10,M10_1,1,10,2041,4,22,1,10
11,M10_1,1,10,2041,4,28,2,10
49,M10_1,1,10,2042,4,4,2,10


In [139]:
display( HTML( BCAprDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
13,M100_169,169,100,2041,4,1,2,100
14,M100_169,169,100,2041,4,5,3,100
15,M100_169,169,100,2041,4,10,3,100
16,M100_169,169,100,2041,4,14,2,100
17,M100_169,169,100,2041,4,18,4,100


In [140]:
LOAprPCKF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOAprDF.to_pickle( LOAprPCKF )
BCAprPCKF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCAprDF.to_pickle( BCAprPCKF )

In [141]:
LOAprDF = LOAprDF.reset_index()
LOAprFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOAprDF.to_feather( LOAprFeatherF )
BCAprDF = BCAprDF.reset_index()
BCAprFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCAprDF.to_feather( BCAprFeatherF )

Depths

In [142]:
LOAprDF = LOWetDF[LOWetDF['Month'] == 4].copy()
BCAprDF = BCWetDF[BCWetDF['Month'] == 4].copy()

In [143]:
display( HTML( LOAprDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
8,M10_1,1,10,2041,4,9,2,2.679129,1.855426,0.823703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,1,10,2041,4,17,2,1.338323,1.121863,0.216460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,M10_1,1,10,2041,4,22,1,1.041847,1.041847,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,M10_1,1,10,2041,4,28,2,7.975859,7.734952,0.240907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,M10_1,1,10,2042,4,4,2,1.162629,0.603097,0.559531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
LOAprMaxCnt = LOAprDF['Wet_Count'].max()
LOAprMaxCnt

24

In [145]:
BCAprMaxCnt = BCAprDF['Wet_Count'].max()
BCAprMaxCnt

44

In [146]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [147]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [148]:
for jJ in range(LOAprMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [149]:
for jJ in range(BCAprMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [150]:
LOAprDF = LOAprDF[LOCurCols].copy()
BCAprDF = BCAprDF[BCCurCols].copy()

In [151]:
LOAprDF.reset_index(drop=True, inplace=True)
BCAprDF.reset_index(drop=True, inplace=True)

In [152]:
display( HTML( LOAprDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22
0,M10_1,4,9,2,2.679129,1.855426,0.823703,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,4,17,2,1.338323,1.121863,0.216460,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,4,22,1,1.041847,1.041847,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,4,28,2,7.975859,7.734952,0.240907,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,4,4,2,1.162629,0.603097,0.559531,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,4,14,3,15.007142,6.463991,7.725245,0.817906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,4,18,2,13.664711,7.645225,6.019485,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,4,23,2,5.677769,3.720771,1.956998,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,4,26,1,1.740083,1.740083,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,4,15,1,0.609208,0.609208,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
display( HTML( BCAprDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42
0,M100_169,4,1,2,1.117730,0.760784,0.356946,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,4,5,3,6.247080,0.301187,4.884895,1.060998,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,4,10,3,8.656114,2.574944,3.531423,2.549747,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,4,14,2,3.402239,2.647642,0.754597,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,4,18,4,2.006652,0.996816,0.287300,0.459019,0.263517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,4,23,4,44.269707,0.217461,20.095345,20.203283,3.753618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,4,29,2,0.751503,0.510895,0.240608,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,4,1,2,0.679566,0.398342,0.281224,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,4,6,1,1.384624,1.384624,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,4,12,2,0.534242,0.275013,0.259229,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
LOAprTotDays = LOAprDF['Wet_Count'].sum()
LOAprTotDays

2707288

In [155]:
BCAprTotDays = BCAprDF['Wet_Count'].sum()
BCAprTotDays

1938689

Setup the arrays that will use to store all of the wet day precipitation depths

In [156]:
GridIDs = np.zeros( LOAprTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOAprTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOAprTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [157]:
LONumAprWet = len( LOAprDF )
LONumAprWet

1188854

In [158]:
iCnt = 0
for iI in range(LONumAprWet):
    cRow = LOAprDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [159]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [160]:
PDDepth.min(), PDDepth.max()

(0.20000735, 271.971)

In [161]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOAprDayDF = pd.DataFrame( data=DataDict )

In [162]:
display( HTML( LOAprDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.707288e+06,2.707288e+06,2.707288e+06
mean,8.662690e+01,3.166969e+01,7.478125e+00
std,4.848505e+01,1.834533e+01,1.291943e+01
min,1.000000e+00,1.000000e+00,2.000073e-01
25%,4.500000e+01,1.500000e+01,1.000000e+00
50%,8.800000e+01,3.200000e+01,3.000000e+00
75%,1.290000e+02,4.700000e+01,7.889037e+00
max,1.680000e+02,6.400000e+01,2.719710e+02


Now want to save the Apr DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [163]:
LOAprPCKF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOAprDayDF.to_pickle( LOAprPCKF )
LOAprFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOAprDayDF.to_feather( LOAprFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [164]:
GridIDs = np.zeros( BCAprTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCAprTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCAprTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [165]:
BCNumAprWet = len( BCAprDF )
BCNumAprWet

815054

In [166]:
iCnt = 0
for iI in range(BCNumAprWet):
    cRow = BCAprDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [167]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [168]:
PDDepth.min(), PDDepth.max()

(0.2000026, 501.98083)

In [169]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCAprDayDF = pd.DataFrame( data=DataDict )

In [170]:
display( HTML( BCAprDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.938689e+06,1.938689e+06,1.938689e+06
mean,1.894035e+02,1.292806e+02,5.925007e+00
std,1.210011e+01,3.836958e+01,1.190094e+01
min,1.690000e+02,6.500000e+01,2.000026e-01
25%,1.790000e+02,9.500000e+01,1.000000e+00
50%,1.890000e+02,1.280000e+02,2.000000e+00
75%,2.000000e+02,1.630000e+02,5.387675e+00
max,2.100000e+02,1.960000e+02,5.019808e+02


Now want to save the Apr DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [171]:
BCAprPCKF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCAprDayDF.to_pickle( BCAprPCKF )
BCAprFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Apr_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCAprDayDF.to_feather( BCAprFeatherF )

### May

Counts

In [172]:
LOMayDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 5].copy()

In [173]:
display( HTML( LOMayDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
12,M10_1,1,10,2041,5,2,2
13,M10_1,1,10,2041,5,7,1
14,M10_1,1,10,2041,5,25,5
54,M10_1,1,10,2042,5,1,1
55,M10_1,1,10,2042,5,3,2


In [174]:
BCMayDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 5].copy()

In [175]:
display( HTML( BCMayDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
20,M100_169,169,100,2041,5,2,5
21,M100_169,169,100,2041,5,8,2
22,M100_169,169,100,2041,5,11,2
23,M100_169,169,100,2041,5,14,1
24,M100_169,169,100,2041,5,16,1


In [176]:
LOMayDF['Grid_Id'] = LOMayDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOMayDF['Model_Id'] = LOMayDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCMayDF['Grid_Id'] = BCMayDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCMayDF['Model_Id'] = BCMayDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [177]:
display( HTML( LOMayDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
12,M10_1,1,10,2041,5,2,2,10
13,M10_1,1,10,2041,5,7,1,10
14,M10_1,1,10,2041,5,25,5,10
54,M10_1,1,10,2042,5,1,1,10
55,M10_1,1,10,2042,5,3,2,10


In [178]:
display( HTML( BCMayDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
20,M100_169,169,100,2041,5,2,5,100
21,M100_169,169,100,2041,5,8,2,100
22,M100_169,169,100,2041,5,11,2,100
23,M100_169,169,100,2041,5,14,1,100
24,M100_169,169,100,2041,5,16,1,100


In [179]:
LOMayPCKF = os.path.normpath( os.path.join( OUT_DIR, "May_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOMayDF.to_pickle( LOMayPCKF )
BCMayPCKF = os.path.normpath( os.path.join( OUT_DIR, "May_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCMayDF.to_pickle( BCMayPCKF )

In [180]:
LOMayDF = LOMayDF.reset_index()
LOMayFeatherF = os.path.normpath( os.path.join( OUT_DIR, "May_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOMayDF.to_feather( LOMayFeatherF )
BCMayDF = BCMayDF.reset_index()
BCMayFeatherF = os.path.normpath( os.path.join( OUT_DIR, "May_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCMayDF.to_feather( BCMayFeatherF )

Depths

In [181]:
LOMayDF = LOWetDF[LOWetDF['Month'] == 5].copy()
BCMayDF = BCWetDF[BCWetDF['Month'] == 5].copy()

In [182]:
display( HTML( LOMayDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
12,M10_1,1,10,2041,5,2,2,1.517800,0.882181,0.635619,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,M10_1,1,10,2041,5,7,1,0.301834,0.301834,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,M10_1,1,10,2041,5,25,5,8.215682,0.341224,1.372592,1.045295,4.612056,0.844515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,M10_1,1,10,2042,5,1,1,0.303284,0.303284,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,M10_1,1,10,2042,5,3,2,0.879973,0.247557,0.632416,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
LOMayMaxCnt = LOMayDF['Wet_Count'].max()
LOMayMaxCnt

38

In [184]:
BCMayMaxCnt = BCMayDF['Wet_Count'].max()
BCMayMaxCnt

70

In [185]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [186]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [187]:
for jJ in range(LOMayMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [188]:
for jJ in range(BCMayMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [189]:
LOMayDF = LOMayDF[LOCurCols].copy()
BCMayDF = BCMayDF[BCCurCols].copy()

In [190]:
LOMayDF.reset_index(drop=True, inplace=True)
BCMayDF.reset_index(drop=True, inplace=True)

In [191]:
display( HTML( LOMayDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36
0,M10_1,5,2,2,1.517800,0.882181,0.635619,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,5,7,1,0.301834,0.301834,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,5,25,5,8.215682,0.341224,1.372592,1.045295,4.612056,0.844515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,5,1,1,0.303284,0.303284,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,5,3,2,0.879973,0.247557,0.632416,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,5,9,2,1.174240,0.933861,0.240379,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,5,16,2,3.718955,1.557190,2.161765,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,5,20,2,0.903627,0.678416,0.225211,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,5,24,1,0.438232,0.438232,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,5,1,5,3.875242,0.269586,0.200927,1.373991,1.562374,0.468365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
display( HTML( BCMayDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47,Day_48,Day_49,Day_50,Day_51,Day_52,Day_53,Day_54,Day_55,Day_56,Day_57,Day_58,Day_59,Day_60,Day_61,Day_62,Day_63,Day_64,Day_65,Day_66,Day_67,Day_68
0,M100_169,5,2,5,22.238921,4.621812,0.476286,1.983641,10.215220,4.941962,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,5,8,2,0.776006,0.238836,0.537169,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,5,11,2,7.597373,2.414795,5.182579,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,5,14,1,0.332055,0.332055,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,5,16,1,0.434535,0.434535,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,5,18,3,1.680134,1.008719,0.333773,0.337641,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,5,23,5,11.741575,0.332798,0.508300,1.945340,3.235819,5.719319,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,5,29,3,9.630340,0.397464,8.734113,0.498764,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,5,5,1,0.244333,0.244333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,5,9,7,35.975456,7.679242,8.251684,3.545091,1.007198,14.390739,0.527915,0.573587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
LOMayTotDays = LOMayDF['Wet_Count'].sum()
LOMayTotDays

3680435

In [194]:
BCMayTotDays = BCMayDF['Wet_Count'].sum()
BCMayTotDays

2642628

Setup the arrays that will use to store all of the wet day precipitation depths

In [195]:
GridIDs = np.zeros( LOMayTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOMayTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOMayTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [196]:
LONumMayWet = len( LOMayDF )
LONumMayWet

1382065

In [197]:
iCnt = 0
for iI in range(LONumMayWet):
    cRow = LOMayDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [198]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [199]:
PDDepth.min(), PDDepth.max()

(0.20000112, 613.24756)

In [200]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOMayDayDF = pd.DataFrame( data=DataDict )

In [201]:
display( HTML( LOMayDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,3.680435e+06,3.680435e+06,3.680435e+06
mean,8.804879e+01,3.175150e+01,8.265864e+00
std,4.825673e+01,1.852611e+01,1.519107e+01
min,1.000000e+00,1.000000e+00,2.000011e-01
25%,4.700000e+01,1.500000e+01,1.137496e+00
50%,9.000000e+01,3.200000e+01,3.000000e+00
75%,1.300000e+02,4.700000e+01,8.512711e+00
max,1.680000e+02,6.400000e+01,6.132476e+02


Now want to save the May DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [202]:
LOMayPCKF = os.path.normpath( os.path.join( OUT_DIR, "May_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOMayDayDF.to_pickle( LOMayPCKF )
LOMayFeatherF = os.path.normpath( os.path.join( OUT_DIR, "May_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOMayDayDF.to_feather( LOMayFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [203]:
GridIDs = np.zeros( BCMayTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCMayTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCMayTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [204]:
BCNumMayWet = len( BCMayDF )
BCNumMayWet

871701

In [205]:
iCnt = 0
for iI in range(BCNumMayWet):
    cRow = BCMayDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [206]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [207]:
PDDepth.min(), PDDepth.max()

(0.2000014, 670.2066)

In [208]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCMayDayDF = pd.DataFrame( data=DataDict )

In [209]:
display( HTML( BCMayDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.642628e+06,2.642628e+06,2.642628e+06
mean,1.894196e+02,1.302409e+02,7.470005e+00
std,1.209373e+01,3.880433e+01,1.551764e+01
min,1.690000e+02,6.500000e+01,2.000014e-01
25%,1.790000e+02,9.500000e+01,1.000000e+00
50%,1.890000e+02,1.300000e+02,3.000000e+00
75%,2.000000e+02,1.640000e+02,7.000000e+00
max,2.100000e+02,1.960000e+02,6.702066e+02


Now want to save the May DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [210]:
BCMayPCKF = os.path.normpath( os.path.join( OUT_DIR, "May_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCMayDayDF.to_pickle( BCMayPCKF )
BCMayFeatherF = os.path.normpath( os.path.join( OUT_DIR, "May_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCMayDayDF.to_feather( BCMayFeatherF )

### Jun

Counts

In [211]:
LOJunDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 6].copy()

In [212]:
display( HTML( LOJunDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
15,M10_1,1,10,2041,6,7,7
16,M10_1,1,10,2041,6,16,1
17,M10_1,1,10,2041,6,20,1
60,M10_1,1,10,2042,6,3,3
61,M10_1,1,10,2042,6,19,1


In [213]:
BCJunDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 6].copy()

In [214]:
display( HTML( BCJunDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
28,M100_169,169,100,2041,6,11,1
29,M100_169,169,100,2041,6,13,1
30,M100_169,169,100,2041,6,25,1
31,M100_169,169,100,2041,6,29,1
87,M100_169,169,100,2042,6,1,1


In [215]:
LOJunDF['Grid_Id'] = LOJunDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOJunDF['Model_Id'] = LOJunDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCJunDF['Grid_Id'] = BCJunDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCJunDF['Model_Id'] = BCJunDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [216]:
display( HTML( LOJunDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
15,M10_1,1,10,2041,6,7,7,10
16,M10_1,1,10,2041,6,16,1,10
17,M10_1,1,10,2041,6,20,1,10
60,M10_1,1,10,2042,6,3,3,10
61,M10_1,1,10,2042,6,19,1,10


In [217]:
display( HTML( BCJunDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
28,M100_169,169,100,2041,6,11,1,100
29,M100_169,169,100,2041,6,13,1,100
30,M100_169,169,100,2041,6,25,1,100
31,M100_169,169,100,2041,6,29,1,100
87,M100_169,169,100,2042,6,1,1,100


In [218]:
LOJunPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOJunDF.to_pickle( LOJunPCKF )
BCJunPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCJunDF.to_pickle( BCJunPCKF )

In [219]:
LOJunDF = LOJunDF.reset_index()
LOJunFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOJunDF.to_feather( LOJunFeatherF )
BCJunDF = BCJunDF.reset_index()
BCJunFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCJunDF.to_feather( BCJunFeatherF )

Depths

In [220]:
LOJunDF = LOWetDF[LOWetDF['Month'] == 6].copy()
BCJunDF = BCWetDF[BCWetDF['Month'] == 6].copy()

In [221]:
display( HTML( LOJunDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
15,M10_1,1,10,2041,6,7,7,129.127991,13.916976,52.227608,9.627274,7.007901,22.957016,19.050659,4.340558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,M10_1,1,10,2041,6,16,1,0.277132,0.277132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,M10_1,1,10,2041,6,20,1,5.667489,5.667489,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,M10_1,1,10,2042,6,3,3,35.543210,7.044112,22.060326,6.438772,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,M10_1,1,10,2042,6,19,1,0.360451,0.360451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
LOJunMaxCnt = LOJunDF['Wet_Count'].max()
LOJunMaxCnt

37

In [223]:
BCJunMaxCnt = BCJunDF['Wet_Count'].max()
BCJunMaxCnt

59

In [224]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [225]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [226]:
for jJ in range(LOJunMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [227]:
for jJ in range(BCJunMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [228]:
LOJunDF = LOJunDF[LOCurCols].copy()
BCJunDF = BCJunDF[BCCurCols].copy()

In [229]:
LOJunDF.reset_index(drop=True, inplace=True)
BCJunDF.reset_index(drop=True, inplace=True)

In [230]:
display( HTML( LOJunDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35
0,M10_1,6,7,7,129.127991,13.916976,52.227608,9.627274,7.007901,22.957016,19.050659,4.340558,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,6,16,1,0.277132,0.277132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,6,20,1,5.667489,5.667489,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,6,3,3,35.543210,7.044112,22.060326,6.438772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,6,19,1,0.360451,0.360451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,6,12,1,0.794325,0.794325,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,6,2,8,51.642311,5.760402,17.685511,1.425108,2.183386,2.400075,16.260826,5.175221,0.751783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,6,12,3,15.436455,9.388186,4.824960,1.223310,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,6,12,1,6.801912,6.801912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,6,16,4,26.553531,4.966498,19.751934,1.623981,0.211118,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [231]:
display( HTML( BCJunDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47,Day_48,Day_49,Day_50,Day_51,Day_52,Day_53,Day_54,Day_55,Day_56,Day_57
0,M100_169,6,11,1,1.797591,1.797591,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,6,13,1,0.316956,0.316956,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,6,25,1,0.570345,0.570345,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,6,29,1,0.308236,0.308236,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,6,1,1,0.253135,0.253135,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,6,5,2,1.027710,0.603317,0.424393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,6,10,1,0.563797,0.563797,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,6,12,1,0.574056,0.574056,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,6,20,2,11.330907,1.943245,9.387663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,6,23,1,0.879900,0.879900,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
LOJunTotDays = LOJunDF['Wet_Count'].sum()
LOJunTotDays

3476029

In [233]:
BCJunTotDays = BCJunDF['Wet_Count'].sum()
BCJunTotDays

2304950

Setup the arrays that will use to store all of the wet day precipitation depths

In [234]:
GridIDs = np.zeros( LOJunTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOJunTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOJunTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [235]:
LONumJunWet = len( LOJunDF )
LONumJunWet

1175795

In [236]:
iCnt = 0
for iI in range(LONumJunWet):
    cRow = LOJunDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [237]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [238]:
PDDepth.min(), PDDepth.max()

(0.20000048, 598.6154)

In [239]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOJunDayDF = pd.DataFrame( data=DataDict )

In [240]:
display( HTML( LOJunDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,3.476029e+06,3.476029e+06,3.476029e+06
mean,8.617216e+01,3.215072e+01,8.875427e+00
std,4.837465e+01,1.842298e+01,1.753530e+01
min,1.000000e+00,1.000000e+00,2.000005e-01
25%,4.400000e+01,1.600000e+01,1.346323e+00
50%,8.700000e+01,3.300000e+01,3.383638e+00
75%,1.280000e+02,4.800000e+01,8.667181e+00
max,1.680000e+02,6.400000e+01,5.986154e+02


Now want to save the Jun DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [241]:
LOJunPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOJunDayDF.to_pickle( LOJunPCKF )
LOJunFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOJunDayDF.to_feather( LOJunFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [242]:
GridIDs = np.zeros( BCJunTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCJunTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCJunTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [243]:
BCNumJunWet = len( BCJunDF )
BCNumJunWet

811311

In [244]:
iCnt = 0
for iI in range(BCNumJunWet):
    cRow = BCJunDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [245]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [246]:
PDDepth.min(), PDDepth.max()

(0.20000628, 730.38544)

In [247]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCJunDayDF = pd.DataFrame( data=DataDict )

In [248]:
display( HTML( BCJunDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.304950e+06,2.304950e+06,2.304950e+06
mean,1.895659e+02,1.294371e+02,7.626701e+00
std,1.206263e+01,3.812561e+01,1.761709e+01
min,1.690000e+02,6.500000e+01,2.000063e-01
25%,1.790000e+02,9.600000e+01,1.000000e+00
50%,1.900000e+02,1.280000e+02,2.624789e+00
75%,2.000000e+02,1.630000e+02,6.797155e+00
max,2.100000e+02,1.960000e+02,7.303854e+02


Now want to save the Jun DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [249]:
BCJunPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCJunDayDF.to_pickle( BCJunPCKF )
BCJunFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jun_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCJunDayDF.to_feather( BCJunFeatherF )

### Jul

Counts

In [250]:
LOJulDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 7].copy()

In [251]:
display( HTML( LOJulDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
18,M10_1,1,10,2041,7,4,2
19,M10_1,1,10,2041,7,21,2
20,M10_1,1,10,2041,7,27,2
62,M10_1,1,10,2042,7,25,1
63,M10_1,1,10,2042,7,27,1


In [252]:
BCJulDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 7].copy()

In [253]:
display( HTML( BCJulDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
32,M100_169,169,100,2041,7,4,2
33,M100_169,169,100,2041,7,7,3
34,M100_169,169,100,2041,7,17,1
35,M100_169,169,100,2041,7,25,1
36,M100_169,169,100,2041,7,28,1


In [254]:
LOJulDF['Grid_Id'] = LOJulDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOJulDF['Model_Id'] = LOJulDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCJulDF['Grid_Id'] = BCJulDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCJulDF['Model_Id'] = BCJulDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [255]:
display( HTML( LOJulDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
18,M10_1,1,10,2041,7,4,2,10
19,M10_1,1,10,2041,7,21,2,10
20,M10_1,1,10,2041,7,27,2,10
62,M10_1,1,10,2042,7,25,1,10
63,M10_1,1,10,2042,7,27,1,10


In [256]:
display( HTML( BCJulDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
32,M100_169,169,100,2041,7,4,2,100
33,M100_169,169,100,2041,7,7,3,100
34,M100_169,169,100,2041,7,17,1,100
35,M100_169,169,100,2041,7,25,1,100
36,M100_169,169,100,2041,7,28,1,100


In [257]:
LOJulPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOJulDF.to_pickle( LOJulPCKF )
BCJulPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCJulDF.to_pickle( BCJulPCKF )

In [258]:
LOJulDF = LOJulDF.reset_index()
LOJulFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOJulDF.to_feather( LOJulFeatherF )
BCJulDF = BCJulDF.reset_index()
BCJulFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCJulDF.to_feather( BCJulFeatherF )

Depths

In [259]:
LOJulDF = LOWetDF[LOWetDF['Month'] == 7].copy()
BCJulDF = BCWetDF[BCWetDF['Month'] == 7].copy()

In [260]:
display( HTML( LOJulDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
18,M10_1,1,10,2041,7,4,2,15.737713,12.258509,3.479204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,M10_1,1,10,2041,7,21,2,9.101020,0.358130,8.742889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,M10_1,1,10,2041,7,27,2,2.370986,1.376068,0.994918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,M10_1,1,10,2042,7,25,1,0.423497,0.423497,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,M10_1,1,10,2042,7,27,1,1.308138,1.308138,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [261]:
LOJulMaxCnt = LOJulDF['Wet_Count'].max()
LOJulMaxCnt

31

In [262]:
BCJulMaxCnt = BCJulDF['Wet_Count'].max()
BCJulMaxCnt

47

In [263]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [264]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [265]:
for jJ in range(LOJulMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [266]:
for jJ in range(BCJulMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [267]:
LOJulDF = LOJulDF[LOCurCols].copy()
BCJulDF = BCJulDF[BCCurCols].copy()

In [268]:
LOJulDF.reset_index(drop=True, inplace=True)
BCJulDF.reset_index(drop=True, inplace=True)

In [269]:
display( HTML( LOJulDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29
0,M10_1,7,4,2,15.737713,12.258509,3.479204,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,7,21,2,9.101020,0.358130,8.742889,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,7,27,2,2.370986,1.376068,0.994918,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,7,25,1,0.423497,0.423497,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,7,27,1,1.308138,1.308138,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,7,31,2,2.030195,1.541180,0.489015,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,7,4,3,4.749729,0.212043,0.219767,4.317918,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,7,22,4,12.566737,0.730478,4.323084,4.604024,2.90915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,7,20,1,1.238079,1.238079,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,7,21,2,15.920345,15.636629,0.283716,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [270]:
display( HTML( BCJulDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45
0,M100_169,7,4,2,1.325400,0.735374,0.590026,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,7,7,3,11.597304,0.408988,5.125972,6.062345,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,7,17,1,0.289327,0.289327,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,7,25,1,3.339192,3.339192,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,7,28,1,0.334490,0.334490,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,7,5,4,17.212297,8.061775,4.550085,4.118289,0.482149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,7,10,1,2.436112,2.436112,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,7,19,1,0.225179,0.225179,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,7,23,1,0.462634,0.462634,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,7,25,2,2.531408,0.749206,1.782201,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [271]:
LOJulTotDays = LOJulDF['Wet_Count'].sum()
LOJulTotDays

2597096

In [272]:
BCJulTotDays = BCJulDF['Wet_Count'].sum()
BCJulTotDays

2260786

Setup the arrays that will use to store all of the wet day precipitation depths

In [273]:
GridIDs = np.zeros( LOJulTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOJulTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOJulTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [274]:
LONumJulWet = len( LOJulDF )
LONumJulWet

989258

In [275]:
iCnt = 0
for iI in range(LONumJulWet):
    cRow = LOJulDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [276]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [277]:
PDDepth.min(), PDDepth.max()

(0.2000001, 786.20105)

In [278]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOJulDayDF = pd.DataFrame( data=DataDict )

In [279]:
display( HTML( LOJulDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.597096e+06,2.597096e+06,2.597096e+06
mean,8.476879e+01,3.227599e+01,9.804311e+00
std,4.861292e+01,1.839010e+01,2.246955e+01
min,1.000000e+00,1.000000e+00,2.000001e-01
25%,4.200000e+01,1.600000e+01,1.014621e+00
50%,8.500000e+01,3.200000e+01,3.000000e+00
75%,1.270000e+02,4.800000e+01,8.788843e+00
max,1.680000e+02,6.400000e+01,7.862010e+02


Now want to save the Jul DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [280]:
LOJulPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOJulDayDF.to_pickle( LOJulPCKF )
LOJulFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOJulDayDF.to_feather( LOJulFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [281]:
GridIDs = np.zeros( BCJulTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCJulTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCJulTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [282]:
BCNumJulWet = len( BCJulDF )
BCNumJulWet

842033

In [283]:
iCnt = 0
for iI in range(BCNumJulWet):
    cRow = BCJulDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [284]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [285]:
PDDepth.min(), PDDepth.max()

(0.2000014, 771.7484)

In [286]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCJulDayDF = pd.DataFrame( data=DataDict )

In [287]:
display( HTML( BCJulDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.260786e+06,2.260786e+06,2.260786e+06
mean,1.899785e+02,1.288513e+02,6.803998e+00
std,1.205567e+01,3.794231e+01,1.746022e+01
min,1.690000e+02,6.500000e+01,2.000014e-01
25%,1.800000e+02,9.600000e+01,1.000000e+00
50%,1.900000e+02,1.280000e+02,2.039097e+00
75%,2.000000e+02,1.610000e+02,5.846856e+00
max,2.100000e+02,1.960000e+02,7.717484e+02


Now want to save the Jul DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [288]:
BCJulPCKF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCJulDayDF.to_pickle( BCJulPCKF )
BCJulFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Jul_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCJulDayDF.to_feather( BCJulFeatherF )

### Aug

Counts

In [289]:
LOAugDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 8].copy()

In [290]:
display( HTML( LOAugDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
21,M10_1,1,10,2041,8,16,2
22,M10_1,1,10,2041,8,28,4
65,M10_1,1,10,2042,8,3,1
66,M10_1,1,10,2042,8,24,16
94,M10_1,1,10,2043,8,3,1


In [291]:
BCAugDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 8].copy()

In [292]:
display( HTML( BCAugDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
37,M100_169,169,100,2041,8,2,3
38,M100_169,169,100,2041,8,6,1
39,M100_169,169,100,2041,8,9,2
40,M100_169,169,100,2041,8,12,1
41,M100_169,169,100,2041,8,16,5


In [293]:
LOAugDF['Grid_Id'] = LOAugDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOAugDF['Model_Id'] = LOAugDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCAugDF['Grid_Id'] = BCAugDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCAugDF['Model_Id'] = BCAugDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [294]:
display( HTML( LOAugDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
21,M10_1,1,10,2041,8,16,2,10
22,M10_1,1,10,2041,8,28,4,10
65,M10_1,1,10,2042,8,3,1,10
66,M10_1,1,10,2042,8,24,16,10
94,M10_1,1,10,2043,8,3,1,10


In [295]:
display( HTML( BCAugDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
37,M100_169,169,100,2041,8,2,3,100
38,M100_169,169,100,2041,8,6,1,100
39,M100_169,169,100,2041,8,9,2,100
40,M100_169,169,100,2041,8,12,1,100
41,M100_169,169,100,2041,8,16,5,100


In [296]:
LOAugPCKF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOAugDF.to_pickle( LOAugPCKF )
BCAugPCKF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCAugDF.to_pickle( BCAugPCKF )

In [297]:
LOAugDF = LOAugDF.reset_index()
LOAugFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOAugDF.to_feather( LOAugFeatherF )
BCAugDF = BCAugDF.reset_index()
BCAugFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCAugDF.to_feather( BCAugFeatherF )

Depths

In [298]:
LOAugDF = LOWetDF[LOWetDF['Month'] == 8].copy()
BCAugDF = BCWetDF[BCWetDF['Month'] == 8].copy()

In [299]:
display( HTML( LOAugDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
21,M10_1,1,10,2041,8,16,2,0.843170,0.638891,0.204278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,M10_1,1,10,2041,8,28,4,10.673820,0.509684,3.660352,3.937469,2.566316,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,M10_1,1,10,2042,8,3,1,5.539091,5.539091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,M10_1,1,10,2042,8,24,16,75.183723,4.730564,26.124653,4.325094,0.727215,0.468614,2.547748,3.738879,6.587192,5.511059,7.29445,2.622775,2.495689,3.579333,0.314916,3.522184,0.593358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,M10_1,1,10,2043,8,3,1,0.361201,0.361201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [300]:
LOAugMaxCnt = LOAugDF['Wet_Count'].max()
LOAugMaxCnt

47

In [301]:
BCAugMaxCnt = BCAugDF['Wet_Count'].max()
BCAugMaxCnt

54

In [302]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [303]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [304]:
for jJ in range(LOAugMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [305]:
for jJ in range(BCAugMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [306]:
LOAugDF = LOAugDF[LOCurCols].copy()
BCAugDF = BCAugDF[BCCurCols].copy()

In [307]:
LOAugDF.reset_index(drop=True, inplace=True)
BCAugDF.reset_index(drop=True, inplace=True)

In [308]:
display( HTML( LOAugDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45
0,M10_1,8,16,2,0.843170,0.638891,0.204278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,8,28,4,10.673820,0.509684,3.660352,3.937469,2.566316,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,8,3,1,5.539091,5.539091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,8,24,16,75.183723,4.730564,26.124653,4.325094,0.727215,0.468614,2.547748,3.738879,6.587192,5.511059,7.29445,2.622775,2.495689,3.579333,0.314916,3.522184,0.593358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,8,3,1,0.361201,0.361201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,8,15,2,4.141936,2.447774,1.694162,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,8,21,2,4.924048,2.459412,2.464636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,8,28,1,2.473803,2.473803,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,8,8,2,4.617188,1.240817,3.376371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,8,15,2,8.427567,5.773272,2.654295,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [309]:
display( HTML( BCAugDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47,Day_48,Day_49,Day_50,Day_51,Day_52
0,M100_169,8,2,3,3.180376,0.246605,1.069437,1.864334,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,8,6,1,1.019987,1.019987,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,8,9,2,2.982629,0.802138,2.180491,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,8,12,1,0.315469,0.315469,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,8,16,5,6.067977,1.413589,3.256916,0.561819,0.453946,0.381707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,8,23,4,3.097758,0.263581,0.764760,1.184793,0.884624,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,8,4,1,0.406082,0.406082,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,8,7,1,0.231329,0.231329,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,8,10,2,1.730999,0.696244,1.034755,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,8,13,1,0.704932,0.704932,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [310]:
LOAugTotDays = LOAugDF['Wet_Count'].sum()
LOAugTotDays

2647474

In [311]:
BCAugTotDays = BCAugDF['Wet_Count'].sum()
BCAugTotDays

2566441

Setup the arrays that will use to store all of the wet day precipitation depths

In [312]:
GridIDs = np.zeros( LOAugTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOAugTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOAugTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [313]:
LONumAugWet = len( LOAugDF )
LONumAugWet

1022133

In [314]:
iCnt = 0
for iI in range(LONumAugWet):
    cRow = LOAugDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [315]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [316]:
PDDepth.min(), PDDepth.max()

(0.20000745, 728.4018)

In [317]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOAugDayDF = pd.DataFrame( data=DataDict )

In [318]:
display( HTML( LOAugDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.647474e+06,2.647474e+06,2.647474e+06
mean,8.761945e+01,3.188172e+01,9.318830e+00
std,4.817121e+01,1.842306e+01,2.002304e+01
min,1.000000e+00,1.000000e+00,2.000075e-01
25%,4.600000e+01,1.600000e+01,1.232978e+00
50%,9.000000e+01,3.200000e+01,3.000000e+00
75%,1.290000e+02,4.800000e+01,8.416030e+00
max,1.680000e+02,6.400000e+01,7.284018e+02


Now want to save the Auguary DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [319]:
LOAugPCKF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOAugDayDF.to_pickle( LOAugPCKF )
LOAugFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOAugDayDF.to_feather( LOAugFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [320]:
GridIDs = np.zeros( BCAugTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCAugTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCAugTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [321]:
BCNumAugWet = len( BCAugDF )
BCNumAugWet

886305

In [322]:
iCnt = 0
for iI in range(BCNumAugWet):
    cRow = BCAugDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [323]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [324]:
PDDepth.min(), PDDepth.max()

(0.20000035, 763.0875)

In [325]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCAugDayDF = pd.DataFrame( data=DataDict )

In [326]:
display( HTML( BCAugDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.566441e+06,2.566441e+06,2.566441e+06
mean,1.898878e+02,1.292364e+02,7.126114e+00
std,1.205546e+01,3.822479e+01,1.803137e+01
min,1.690000e+02,6.500000e+01,2.000003e-01
25%,1.800000e+02,9.600000e+01,1.000000e+00
50%,1.900000e+02,1.280000e+02,2.298769e+00
75%,2.000000e+02,1.620000e+02,6.000000e+00
max,2.100000e+02,1.960000e+02,7.630875e+02


Now want to save the Aug DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [327]:
BCAugPCKF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCAugDayDF.to_pickle( BCAugPCKF )
BCAugFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Aug_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCAugDayDF.to_feather( BCAugFeatherF )

### Sep

Counts

In [328]:
LOSepDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 9].copy()

In [329]:
display( HTML( LOSepDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
23,M10_1,1,10,2041,9,18,7
24,M10_1,1,10,2041,9,29,8
67,M10_1,1,10,2042,9,25,1
98,M10_1,1,10,2043,9,2,2
99,M10_1,1,10,2043,9,9,2


In [330]:
BCSepDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 9].copy()

In [331]:
display( HTML( BCSepDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
43,M100_169,169,100,2041,9,2,3
44,M100_169,169,100,2041,9,8,1
45,M100_169,169,100,2041,9,14,3
46,M100_169,169,100,2041,9,21,4
47,M100_169,169,100,2041,9,27,2


In [332]:
LOSepDF['Grid_Id'] = LOSepDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOSepDF['Model_Id'] = LOSepDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCSepDF['Grid_Id'] = BCSepDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCSepDF['Model_Id'] = BCSepDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [333]:
display( HTML( LOSepDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
23,M10_1,1,10,2041,9,18,7,10
24,M10_1,1,10,2041,9,29,8,10
67,M10_1,1,10,2042,9,25,1,10
98,M10_1,1,10,2043,9,2,2,10
99,M10_1,1,10,2043,9,9,2,10


In [334]:
display( HTML( BCSepDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
43,M100_169,169,100,2041,9,2,3,100
44,M100_169,169,100,2041,9,8,1,100
45,M100_169,169,100,2041,9,14,3,100
46,M100_169,169,100,2041,9,21,4,100
47,M100_169,169,100,2041,9,27,2,100


In [335]:
LOSepPCKF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOSepDF.to_pickle( LOSepPCKF )
BCSepPCKF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCSepDF.to_pickle( BCSepPCKF )

In [336]:
LOSepDF = LOSepDF.reset_index()
LOSepFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOSepDF.to_feather( LOSepFeatherF )
BCSepDF = BCSepDF.reset_index()
BCSepFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCSepDF.to_feather( BCSepFeatherF )

Depths

In [337]:
LOSepDF = LOWetDF[LOWetDF['Month'] == 9].copy()
BCSepDF = BCWetDF[BCWetDF['Month'] == 9].copy()

In [338]:
display( HTML( LOSepDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
23,M10_1,1,10,2041,9,18,7,115.629359,0.351824,41.558170,31.706631,0.782360,15.525678,7.702685,18.00201,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,M10_1,1,10,2041,9,29,8,72.005449,1.014038,17.067812,39.502342,0.358607,7.246845,2.266023,2.29733,2.252451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,M10_1,1,10,2042,9,25,1,1.554924,1.554924,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,M10_1,1,10,2043,9,2,2,13.362334,2.492887,10.869447,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,M10_1,1,10,2043,9,9,2,6.238373,5.279783,0.958590,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [339]:
LOSepMaxCnt = LOSepDF['Wet_Count'].max()
LOSepMaxCnt

33

In [340]:
BCSepMaxCnt = BCSepDF['Wet_Count'].max()
BCSepMaxCnt

38

In [341]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [342]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [343]:
for jJ in range(LOSepMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [344]:
for jJ in range(BCSepMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [345]:
LOSepDF = LOSepDF[LOCurCols].copy()
BCSepDF = BCSepDF[BCCurCols].copy()

In [346]:
LOSepDF.reset_index(drop=True, inplace=True)
BCSepDF.reset_index(drop=True, inplace=True)

In [347]:
display( HTML( LOSepDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,M10_1,9,18,7,115.629359,0.351824,41.558170,31.706631,0.782360,15.525678,7.702685,18.00201,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,9,29,8,72.005449,1.014038,17.067812,39.502342,0.358607,7.246845,2.266023,2.29733,2.252451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,9,25,1,1.554924,1.554924,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,9,2,2,13.362334,2.492887,10.869447,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,9,9,2,6.238373,5.279783,0.958590,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,9,20,1,0.296873,0.296873,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,9,26,2,1.099609,0.377246,0.722362,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,9,8,1,0.707503,0.707503,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,9,21,1,0.525449,0.525449,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,9,23,2,4.595751,1.784329,2.811422,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [348]:
display( HTML( BCSepDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36
0,M100_169,9,2,3,12.349883,0.202921,1.081207,11.065755,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,9,8,1,0.426163,0.426163,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,9,14,3,6.206579,2.042567,2.184682,1.979330,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,9,21,4,17.667694,0.402245,9.044039,7.582579,0.638832,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,9,27,2,7.331701,6.031027,1.300674,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,9,3,17,121.021739,2.351810,6.113557,13.696325,4.538840,1.852905,2.041713,0.675269,0.258611,1.39865,1.420194,17.86031,12.226068,1.839419,0.209042,5.182018,24.12291,25.234098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,9,21,2,11.459327,7.507701,3.951626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,9,24,4,19.241143,3.468113,4.146823,7.584873,4.041334,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,9,29,2,26.859771,22.060274,4.799497,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,9,3,1,0.428505,0.428505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [349]:
LOSepTotDays = LOSepDF['Wet_Count'].sum()
LOSepTotDays

3450150

In [350]:
BCSepTotDays = BCSepDF['Wet_Count'].sum()
BCSepTotDays

2463287

Setup the arrays that will use to store all of the wet day precipitation depths

In [351]:
GridIDs = np.zeros( LOSepTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOSepTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOSepTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [352]:
LONumSepWet = len( LOSepDF )
LONumSepWet

1182589

In [353]:
iCnt = 0
for iI in range(LONumSepWet):
    cRow = LOSepDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [354]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [355]:
PDDepth.min(), PDDepth.max()

(0.20000452, 559.81976)

In [356]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOSepDayDF = pd.DataFrame( data=DataDict )

In [357]:
display( HTML( LOSepDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,3.450150e+06,3.450150e+06,3.450150e+06
mean,8.543508e+01,3.194684e+01,1.130018e+01
std,4.849780e+01,1.848764e+01,2.290721e+01
min,1.000000e+00,1.000000e+00,2.000045e-01
25%,4.300000e+01,1.500000e+01,1.824726e+00
50%,8.600000e+01,3.300000e+01,3.956971e+00
75%,1.270000e+02,4.800000e+01,1.016976e+01
max,1.680000e+02,6.400000e+01,5.598198e+02


Now want to save the Sepuary DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [358]:
LOSepPCKF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOSepDayDF.to_pickle( LOSepPCKF )
LOSepFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOSepDayDF.to_feather( LOSepFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [359]:
GridIDs = np.zeros( BCSepTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCSepTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCSepTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [360]:
BCNumSepWet = len( BCSepDF )
BCNumSepWet

797460

In [361]:
iCnt = 0
for iI in range(BCNumSepWet):
    cRow = BCSepDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [362]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [363]:
PDDepth.min(), PDDepth.max()

(0.20000166, 612.5825)

In [364]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCSepDayDF = pd.DataFrame( data=DataDict )

In [365]:
display( HTML( BCSepDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.463287e+06,2.463287e+06,2.463287e+06
mean,1.895181e+02,1.286933e+02,8.930671e+00
std,1.209695e+01,3.808691e+01,1.936238e+01
min,1.690000e+02,6.500000e+01,2.000017e-01
25%,1.790000e+02,9.500000e+01,1.000000e+00
50%,1.890000e+02,1.270000e+02,3.000000e+00
75%,2.000000e+02,1.620000e+02,8.000000e+00
max,2.100000e+02,1.960000e+02,6.125825e+02


Now want to save the Sep DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [366]:
BCSepPCKF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCSepDayDF.to_pickle( BCSepPCKF )
BCSepFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Sep_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCSepDayDF.to_feather( BCSepFeatherF )

### Oct

Counts

In [367]:
LOOctDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 10].copy()

In [368]:
display( HTML( LOOctDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
25,M10_1,1,10,2041,10,9,1
26,M10_1,1,10,2041,10,12,2
27,M10_1,1,10,2041,10,24,3
68,M10_1,1,10,2042,10,1,2
69,M10_1,1,10,2042,10,11,1


In [369]:
BCOctDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 10].copy()

In [370]:
display( HTML( BCOctDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
48,M100_169,169,100,2041,10,1,1
49,M100_169,169,100,2041,10,7,4
50,M100_169,169,100,2041,10,13,1
51,M100_169,169,100,2041,10,16,2
52,M100_169,169,100,2041,10,19,3


In [371]:
LOOctDF['Grid_Id'] = LOOctDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LOOctDF['Model_Id'] = LOOctDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCOctDF['Grid_Id'] = BCOctDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCOctDF['Model_Id'] = BCOctDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [372]:
display( HTML( LOOctDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
25,M10_1,1,10,2041,10,9,1,10
26,M10_1,1,10,2041,10,12,2,10
27,M10_1,1,10,2041,10,24,3,10
68,M10_1,1,10,2042,10,1,2,10
69,M10_1,1,10,2042,10,11,1,10


In [373]:
display( HTML( BCOctDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
48,M100_169,169,100,2041,10,1,1,100
49,M100_169,169,100,2041,10,7,4,100
50,M100_169,169,100,2041,10,13,1,100
51,M100_169,169,100,2041,10,16,2,100
52,M100_169,169,100,2041,10,19,3,100


In [374]:
LOOctPCKF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LOOctDF.to_pickle( LOOctPCKF )
BCOctPCKF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCOctDF.to_pickle( BCOctPCKF )

In [375]:
LOOctDF = LOOctDF.reset_index()
LOOctFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LOOctDF.to_feather( LOOctFeatherF )
BCOctDF = BCOctDF.reset_index()
BCOctFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCOctDF.to_feather( BCOctFeatherF )

Depths

In [376]:
LOOctDF = LOWetDF[LOWetDF['Month'] == 10].copy()
BCOctDF = BCWetDF[BCWetDF['Month'] == 10].copy()

In [377]:
display( HTML( LOOctDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
25,M10_1,1,10,2041,10,9,1,0.213546,0.213546,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,M10_1,1,10,2041,10,12,2,3.547785,1.653783,1.894002,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,M10_1,1,10,2041,10,24,3,20.763033,4.366429,12.235162,4.161442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,M10_1,1,10,2042,10,1,2,11.954669,6.103601,5.851068,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,M10_1,1,10,2042,10,11,1,2.473134,2.473134,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [378]:
LOOctMaxCnt = LOOctDF['Wet_Count'].max()
LOOctMaxCnt

21

In [379]:
BCOctMaxCnt = BCOctDF['Wet_Count'].max()
BCOctMaxCnt

29

In [380]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [381]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [382]:
for jJ in range(LOOctMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [383]:
for jJ in range(BCOctMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [384]:
LOOctDF = LOOctDF[LOCurCols].copy()
BCOctDF = BCOctDF[BCCurCols].copy()

In [385]:
LOOctDF.reset_index(drop=True, inplace=True)
BCOctDF.reset_index(drop=True, inplace=True)

In [386]:
display( HTML( LOOctDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19
0,M10_1,10,9,1,0.213546,0.213546,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,10,12,2,3.547785,1.653783,1.894002,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,10,24,3,20.763033,4.366429,12.235162,4.161442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,10,1,2,11.954669,6.103601,5.851068,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,10,11,1,2.473134,2.473134,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,10,21,2,2.959794,2.311987,0.647807,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,10,2,1,0.242303,0.242303,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,10,11,1,2.632352,2.632352,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,10,21,3,15.008643,13.720532,0.983606,0.304505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,10,26,2,8.952271,0.358577,8.593694,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [387]:
display( HTML( BCOctDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27
0,M100_169,10,1,1,0.319187,0.319187,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,10,7,4,12.965562,4.507721,3.959408,1.471646,3.026787,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,10,13,1,2.422547,2.422547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,10,16,2,5.895690,1.083629,4.812061,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,10,19,3,6.063480,1.308872,1.299492,3.455117,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,10,25,1,0.399612,0.399612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,10,27,2,6.581759,3.440936,3.140823,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,10,30,6,16.796412,0.323259,5.266560,1.609516,1.780358,3.835179,3.981541,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,10,2,1,0.769648,0.769648,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,10,4,7,63.722428,31.055691,11.254120,9.956639,1.530668,1.190062,0.855890,7.879358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [388]:
LOOctTotDays = LOOctDF['Wet_Count'].sum()
LOOctTotDays

2671562

In [389]:
BCOctTotDays = BCOctDF['Wet_Count'].sum()
BCOctTotDays

1722287

Setup the arrays that will use to store all of the wet day precipitation depths

In [390]:
GridIDs = np.zeros( LOOctTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LOOctTotDays, dtype=np.int32 )
PDDepth = np.zeros( LOOctTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [391]:
LONumOctWet = len( LOOctDF )
LONumOctWet

1082910

In [392]:
iCnt = 0
for iI in range(LONumOctWet):
    cRow = LOOctDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [393]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [394]:
PDDepth.min(), PDDepth.max()

(0.20000033, 539.9713)

In [395]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LOOctDayDF = pd.DataFrame( data=DataDict )

In [396]:
display( HTML( LOOctDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.671562e+06,2.671562e+06,2.671562e+06
mean,8.495488e+01,3.209971e+01,8.994591e+00
std,4.852897e+01,1.842501e+01,1.723425e+01
min,1.000000e+00,1.000000e+00,2.000003e-01
25%,4.300000e+01,1.600000e+01,1.331948e+00
50%,8.500000e+01,3.200000e+01,3.000000e+00
75%,1.270000e+02,4.800000e+01,8.071073e+00
max,1.680000e+02,6.400000e+01,5.399713e+02


Now want to save the Octuary DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [397]:
LOOctPCKF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LOOctDayDF.to_pickle( LOOctPCKF )
LOOctFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LOOctDayDF.to_feather( LOOctFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [398]:
GridIDs = np.zeros( BCOctTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCOctTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCOctTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [399]:
BCNumOctWet = len( BCOctDF )
BCNumOctWet

708693

In [400]:
iCnt = 0
for iI in range(BCNumOctWet):
    cRow = BCOctDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [401]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [402]:
PDDepth.min(), PDDepth.max()

(0.20000169, 319.39478)

In [403]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCOctDayDF = pd.DataFrame( data=DataDict )

In [404]:
display( HTML( BCOctDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.722287e+06,1.722287e+06,1.722287e+06
mean,1.893904e+02,1.293439e+02,7.692143e+00
std,1.207559e+01,3.845319e+01,1.510067e+01
min,1.690000e+02,6.500000e+01,2.000017e-01
25%,1.790000e+02,9.500000e+01,1.000000e+00
50%,1.890000e+02,1.290000e+02,2.561655e+00
75%,2.000000e+02,1.630000e+02,6.880468e+00
max,2.100000e+02,1.960000e+02,3.193948e+02


Now want to save the Oct DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [405]:
BCOctPCKF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCOctDayDF.to_pickle( BCOctPCKF )
BCOctFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Oct_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCOctDayDF.to_feather( BCOctFeatherF )

### Nov

Counts

In [406]:
LONovDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 11].copy()

In [407]:
display( HTML( LONovDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
28,M10_1,1,10,2041,11,4,5
29,M10_1,1,10,2041,11,14,3
30,M10_1,1,10,2041,11,21,3
71,M10_1,1,10,2042,11,19,2
72,M10_1,1,10,2042,11,23,1


In [408]:
BCNovDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 11].copy()

In [409]:
display( HTML( BCNovDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
56,M100_169,169,100,2041,11,6,3
57,M100_169,169,100,2041,11,11,2
58,M100_169,169,100,2041,11,14,1
59,M100_169,169,100,2041,11,18,9
113,M100_169,169,100,2042,11,2,7


In [410]:
LONovDF['Grid_Id'] = LONovDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LONovDF['Model_Id'] = LONovDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCNovDF['Grid_Id'] = BCNovDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCNovDF['Model_Id'] = BCNovDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [411]:
display( HTML( LONovDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
28,M10_1,1,10,2041,11,4,5,10
29,M10_1,1,10,2041,11,14,3,10
30,M10_1,1,10,2041,11,21,3,10
71,M10_1,1,10,2042,11,19,2,10
72,M10_1,1,10,2042,11,23,1,10


In [412]:
display( HTML( BCNovDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
56,M100_169,169,100,2041,11,6,3,100
57,M100_169,169,100,2041,11,11,2,100
58,M100_169,169,100,2041,11,14,1,100
59,M100_169,169,100,2041,11,18,9,100
113,M100_169,169,100,2042,11,2,7,100


In [413]:
LONovPCKF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LONovDF.to_pickle( LONovPCKF )
BCNovPCKF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCNovDF.to_pickle( BCNovPCKF )

In [414]:
LONovDF = LONovDF.reset_index()
LONovFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LONovDF.to_feather( LONovFeatherF )
BCNovDF = BCNovDF.reset_index()
BCNovFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCNovDF.to_feather( BCNovFeatherF )

Depths

In [415]:
LONovDF = LOWetDF[LOWetDF['Month'] == 11].copy()
BCNovDF = BCWetDF[BCWetDF['Month'] == 11].copy()

In [416]:
display( HTML( LONovDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
28,M10_1,1,10,2041,11,4,5,77.605431,23.366665,42.196705,7.561383,3.452138,1.02854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,M10_1,1,10,2041,11,14,3,43.905564,0.309554,27.726692,15.869318,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,M10_1,1,10,2041,11,21,3,2.918117,1.919468,0.642385,0.356264,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,M10_1,1,10,2042,11,19,2,0.917988,0.344351,0.573637,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,M10_1,1,10,2042,11,23,1,0.663770,0.663770,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [417]:
LONovMaxCnt = LONovDF['Wet_Count'].max()
LONovMaxCnt

22

In [418]:
BCNovMaxCnt = BCNovDF['Wet_Count'].max()
BCNovMaxCnt

18

In [419]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [420]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [421]:
for jJ in range(LONovMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [422]:
for jJ in range(BCNovMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [423]:
LONovDF = LONovDF[LOCurCols].copy()
BCNovDF = BCNovDF[BCCurCols].copy()

In [424]:
LONovDF.reset_index(drop=True, inplace=True)
BCNovDF.reset_index(drop=True, inplace=True)

In [425]:
display( HTML( LONovDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20
0,M10_1,11,4,5,77.605431,23.366665,42.196705,7.561383,3.452138,1.02854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,11,14,3,43.905564,0.309554,27.726692,15.869318,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,11,21,3,2.918117,1.919468,0.642385,0.356264,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,11,19,2,0.917988,0.344351,0.573637,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,11,23,1,0.663770,0.663770,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,11,25,2,19.815844,12.352502,7.463342,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,11,2,2,19.332657,17.574478,1.758179,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,11,11,3,9.899924,3.806426,2.491774,3.601723,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,11,29,1,0.205841,0.205841,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,11,26,1,0.352884,0.352884,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [426]:
display( HTML( BCNovDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16
0,M100_169,11,6,3,17.719911,2.268030,6.589195,8.862686,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,11,11,2,12.541041,7.731613,4.809428,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,11,14,1,0.525828,0.525828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,11,18,9,38.176209,4.022158,0.813462,2.974518,11.720989,2.955415,4.706886,9.113164,1.628283,0.241334,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,11,2,7,13.037852,3.730960,2.097119,4.296114,0.693993,0.960670,0.351314,0.907681,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,11,11,2,17.950532,14.821553,3.128979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,11,29,1,0.252803,0.252803,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,11,1,1,0.492714,0.492714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,11,11,1,0.221435,0.221435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,11,19,2,3.849155,0.459956,3.389199,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [427]:
LONovTotDays = LONovDF['Wet_Count'].sum()
LONovTotDays

2271358

In [428]:
BCNovTotDays = BCNovDF['Wet_Count'].sum()
BCNovTotDays

1300848

Setup the arrays that will use to store all of the wet day precipitation depths

In [429]:
GridIDs = np.zeros( LONovTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LONovTotDays, dtype=np.int32 )
PDDepth = np.zeros( LONovTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [430]:
LONumNovWet = len( LONovDF )
LONumNovWet

1074545

In [431]:
iCnt = 0
for iI in range(LONumNovWet):
    cRow = LONovDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [432]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [433]:
PDDepth.min(), PDDepth.max()

(0.20000023, 280.27167)

In [434]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LONovDayDF = pd.DataFrame( data=DataDict )

In [435]:
display( HTML( LONovDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,2.271358e+06,2.271358e+06,2.271358e+06
mean,8.640851e+01,3.175819e+01,5.839855e+00
std,4.863517e+01,1.830187e+01,1.035766e+01
min,1.000000e+00,1.000000e+00,2.000002e-01
25%,4.400000e+01,1.600000e+01,1.000000e+00
50%,8.800000e+01,3.200000e+01,2.137323e+00
75%,1.290000e+02,4.800000e+01,5.417707e+00
max,1.680000e+02,6.400000e+01,2.802717e+02


Now want to save the Novuary DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [436]:
LONovPCKF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LONovDayDF.to_pickle( LONovPCKF )
LONovFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LONovDayDF.to_feather( LONovFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [437]:
GridIDs = np.zeros( BCNovTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCNovTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCNovTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [438]:
BCNumNovWet = len( BCNovDF )
BCNumNovWet

643678

In [439]:
iCnt = 0
for iI in range(BCNumNovWet):
    cRow = BCNovDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [440]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [441]:
PDDepth.min(), PDDepth.max()

(0.20000085, 192.31624)

In [442]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCNovDayDF = pd.DataFrame( data=DataDict )

In [443]:
display( HTML( BCNovDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.300848e+06,1.300848e+06,1.300848e+06
mean,1.893673e+02,1.296460e+02,4.682192e+00
std,1.207899e+01,3.811221e+01,8.045762e+00
min,1.690000e+02,6.500000e+01,2.000009e-01
25%,1.790000e+02,9.600000e+01,1.000000e+00
50%,1.890000e+02,1.290000e+02,2.000000e+00
75%,2.000000e+02,1.620000e+02,4.336049e+00
max,2.100000e+02,1.960000e+02,1.923162e+02


Now want to save the Nov DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [444]:
BCNovPCKF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCNovDayDF.to_pickle( BCNovPCKF )
BCNovFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Nov_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCNovDayDF.to_feather( BCNovFeatherF )

### Dec

Counts

In [445]:
LODecDF = LOWetDF[LORootCols].loc[LOWetDF['Month'] == 12].copy()

In [446]:
display( HTML( LODecDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
31,M10_1,1,10,2041,12,3,4
32,M10_1,1,10,2041,12,10,5
33,M10_1,1,10,2041,12,20,2
34,M10_1,1,10,2041,12,25,1
35,M10_1,1,10,2041,12,28,4


In [447]:
BCDecDF = BCWetDF[BCRootCols].loc[BCWetDF['Month'] == 12].copy()

In [448]:
display( HTML( BCDecDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count
60,M100_169,169,100,2041,12,2,1
61,M100_169,169,100,2041,12,4,1
62,M100_169,169,100,2041,12,14,1
63,M100_169,169,100,2041,12,22,1
116,M100_169,169,100,2042,12,5,1


In [449]:
LODecDF['Grid_Id'] = LODecDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
LODecDF['Model_Id'] = LODecDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )
BCDecDF['Grid_Id'] = BCDecDF.apply( lambda row: ExGridNum(row['MGrid_Id']), axis=1 )
BCDecDF['Model_Id'] = BCDecDF.apply( lambda row: ExModNum(row['MGrid_Id']), axis=1 )

In [450]:
display( HTML( LODecDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
31,M10_1,1,10,2041,12,3,4,10
32,M10_1,1,10,2041,12,10,5,10
33,M10_1,1,10,2041,12,20,2,10
34,M10_1,1,10,2041,12,25,1,10
35,M10_1,1,10,2041,12,28,4,10


In [451]:
display( HTML( BCDecDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Model_Id
60,M100_169,169,100,2041,12,2,1,100
61,M100_169,169,100,2041,12,4,1,100
62,M100_169,169,100,2041,12,14,1,100
63,M100_169,169,100,2041,12,22,1,100
116,M100_169,169,100,2042,12,5,1,100


In [452]:
LODecPCKF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetCnt_CMIP5_LOCA_2041-2070.pickle" ) )
LODecDF.to_pickle( LODecPCKF )
BCDecPCKF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetCnt_CMIP5_BCCA_2041-2070.pickle" ) )
BCDecDF.to_pickle( BCDecPCKF )

In [453]:
LODecDF = LODecDF.reset_index()
LODecFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetCnt_CMIP5_LOCA_2041-2070.feather" ) )
LODecDF.to_feather( LODecFeatherF )
BCDecDF = BCDecDF.reset_index()
BCDecFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetCnt_CMIP5_BCCA_2041-2070.feather" ) )
BCDecDF.to_feather( BCDecFeatherF )

Depths

In [454]:
LODecDF = LOWetDF[LOWetDF['Month'] == 12].copy()
BCDecDF = BCWetDF[BCWetDF['Month'] == 12].copy()

In [455]:
display( HTML( LODecDF.head().to_html() ) )

,MGrid_Id,Grid_Id,Mod_Id,Year,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Day_32,Day_33,Day_34,Day_35,Day_36,Day_37,Day_38,Day_39,Day_40,Day_41,Day_42,Day_43,Day_44,Day_45,Day_46,Day_47
31,M10_1,1,10,2041,12,3,4,4.004980,0.480637,1.955310,1.115839,0.453194,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,M10_1,1,10,2041,12,10,5,12.994454,6.873072,4.170993,0.789437,0.629417,0.531535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,M10_1,1,10,2041,12,20,2,22.882125,10.900208,11.981916,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,M10_1,1,10,2041,12,25,1,0.995051,0.995051,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,M10_1,1,10,2041,12,28,4,45.642715,7.486603,14.831511,20.863621,2.460980,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [456]:
LODecMaxCnt = LODecDF['Wet_Count'].max()
LODecMaxCnt

16

In [457]:
BCDecMaxCnt = BCDecDF['Wet_Count'].max()
BCDecMaxCnt

22

In [458]:
LOCurCols = [LORootCols[0], LORootCols[4], LORootCols[5] ]
LOCurCols

['MGrid_Id', 'Month', 'Day']

In [459]:
BCCurCols = [BCRootCols[0], BCRootCols[4], BCRootCols[5] ]
BCCurCols

['MGrid_Id', 'Month', 'Day']

In [460]:
for jJ in range(LODecMaxCnt):
    LOCurCols.append( LODayCols[jJ] )
# end for

In [461]:
for jJ in range(BCDecMaxCnt):
    BCCurCols.append( BCDayCols[jJ] )
# end for

In [462]:
LODecDF = LODecDF[LOCurCols].copy()
BCDecDF = BCDecDF[BCCurCols].copy()

In [463]:
LODecDF.reset_index(drop=True, inplace=True)
BCDecDF.reset_index(drop=True, inplace=True)

In [464]:
display( HTML( LODecDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14
0,M10_1,12,3,4,4.004980,0.480637,1.955310,1.115839,0.453194,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M10_1,12,10,5,12.994454,6.873072,4.170993,0.789437,0.629417,0.531535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M10_1,12,20,2,22.882125,10.900208,11.981916,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M10_1,12,25,1,0.995051,0.995051,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M10_1,12,28,4,45.642715,7.486603,14.831511,20.863621,2.460980,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M10_1,12,1,4,66.285490,9.206924,25.507774,27.894987,3.675804,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M10_1,12,7,1,1.858607,1.858607,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M10_1,12,12,2,1.073908,0.625345,0.448563,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M10_1,12,16,1,0.349934,0.349934,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M10_1,12,18,1,1.331961,1.331961,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [465]:
display( HTML( BCDecDF.iloc[:10].to_html() ) )

,MGrid_Id,Month,Day,Wet_Count,Total_Depth,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20
0,M100_169,12,2,1,0.801230,0.801230,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M100_169,12,4,1,0.347606,0.347606,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M100_169,12,14,1,0.223261,0.223261,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M100_169,12,22,1,0.921073,0.921073,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M100_169,12,5,1,1.522334,1.522334,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,M100_169,12,27,1,0.921838,0.921838,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M100_169,12,14,2,2.412430,0.607546,1.804884,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,M100_169,12,20,6,8.173888,1.539562,4.108276,1.208865,0.595626,0.386611,0.334949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,M100_169,12,28,1,0.254445,0.254445,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,M100_169,12,31,1,0.588869,0.588869,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [466]:
LODecTotDays = LODecDF['Wet_Count'].sum()
LODecTotDays

1831304

In [467]:
BCDecTotDays = BCDecDF['Wet_Count'].sum()
BCDecTotDays

1228899

Setup the arrays that will use to store all of the wet day precipitation depths

In [468]:
GridIDs = np.zeros( LODecTotDays, dtype=np.int32 )
ModelIDs = np.zeros( LODecTotDays, dtype=np.int32 )
PDDepth = np.zeros( LODecTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [469]:
LONumDecWet = len( LODecDF )
LONumDecWet

990725

In [470]:
iCnt = 0
for iI in range(LONumDecWet):
    cRow = LODecDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[LOCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [471]:
GridIDs.min(), GridIDs.max()

(1, 168)

In [472]:
PDDepth.min(), PDDepth.max()

(0.20000188, 501.23065)

In [473]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
LODecDayDF = pd.DataFrame( data=DataDict )

In [474]:
display( HTML( LODecDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.831304e+06,1.831304e+06,1.831304e+06
mean,8.693449e+01,3.187757e+01,4.657842e+00
std,4.905092e+01,1.854190e+01,9.364170e+00
min,1.000000e+00,1.000000e+00,2.000019e-01
25%,4.400000e+01,1.600000e+01,1.000000e+00
50%,8.900000e+01,3.200000e+01,2.000000e+00
75%,1.300000e+02,4.800000e+01,4.000000e+00
max,1.680000e+02,6.400000e+01,5.012307e+02


Now want to save the Dec DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [475]:
LODecPCKF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetDep_CMIP5_LOCA_2041-2070.pickle" ) )
LODecDayDF.to_pickle( LODecPCKF )
LODecFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetDep_CMIP5_LOCA_2041-2070.feather" ) )
LODecDayDF.to_feather( LODecFeatherF )

Now BCCA, setup the arrays that will use to store all of the wet day precipitation depths

In [476]:
GridIDs = np.zeros( BCDecTotDays, dtype=np.int32 )
ModelIDs = np.zeros( BCDecTotDays, dtype=np.int32 )
PDDepth = np.zeros( BCDecTotDays, dtype=np.float32 )
MGridIDs = [ "%d" % x for x in GridIDs.tolist() ]

In [477]:
BCNumDecWet = len( BCDecDF )
BCNumDecWet

642685

In [478]:
iCnt = 0
for iI in range(BCNumDecWet):
    cRow = BCDecDF.loc[iI]
    cMGridId = cRow.at["MGrid_Id"]
    cNumWet = int( cRow.at["Wet_Count"] )
    cGridId = ExGridNum( cMGridId )
    cModelId = ExModNum( cMGridId )
    for jJ in range(cNumWet):
        cPDep = float( cRow.at[BCCurCols[StartInd+jJ]] )
        MGridIDs[iCnt] = cMGridId
        GridIDs[iCnt] = cGridId
        ModelIDs[iCnt] = cModelId
        PDDepth[iCnt] = cPDep
        iCnt += 1
    # end of inner for
# end of outer for

In [479]:
GridIDs.min(), GridIDs.max()

(169, 210)

In [480]:
PDDepth.min(), PDDepth.max()

(0.20000152, 197.13266)

In [481]:
DataDict = { "MGrid_Id" : MGridIDs,
             "Grid_Id" : GridIDs,
             "Model_Id" : ModelIDs,
             "Precip_mm" : PDDepth, }
BCDecDayDF = pd.DataFrame( data=DataDict )

In [482]:
display( HTML( BCDecDayDF.describe().to_html() ))

,Grid_Id,Model_Id,Precip_mm
count,1.228899e+06,1.228899e+06,1.228899e+06
mean,1.894644e+02,1.298767e+02,3.927784e+00
std,1.209007e+01,3.872053e+01,6.570578e+00
min,1.690000e+02,6.500000e+01,2.000015e-01
25%,1.790000e+02,9.500000e+01,1.000000e+00
50%,1.890000e+02,1.300000e+02,2.000000e+00
75%,2.000000e+02,1.640000e+02,4.000000e+00
max,2.100000e+02,1.960000e+02,1.971327e+02


Now want to save the Decuary DataFrame as both a pickle and a feather so that can reuse it with both Python and R.

In [483]:
BCDecPCKF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetDep_CMIP5_BCCA_2041-2070.pickle" ) )
BCDecDayDF.to_pickle( BCDecPCKF )
BCDecFeatherF = os.path.normpath( os.path.join( OUT_DIR, "Dec_WetDep_CMIP5_BCCA_2041-2070.feather" ) )
BCDecDayDF.to_feather( BCDecFeatherF )